# Setting up IRENA dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

Uploading the main dataframe "REinv" as in Renewable Energy Investments

In [4]:
REinv = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/IRENA_RE_Public_Investment_Database_Extract_November19.csv')

In [5]:
REinv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9275 entries, 0 to 9274
Data columns (total 10 columns):
Recipient Country/Area    9275 non-null object
ISO-code                  9275 non-null object
Region                    9275 non-null object
Project                   9023 non-null object
Year                      9275 non-null int64
Investor                  9275 non-null object
Technology                9275 non-null object
Asset Class               9275 non-null object
Amount (USD million)      9275 non-null object
Source                    9275 non-null object
dtypes: int64(1), object(9)
memory usage: 724.7+ KB


It is fortunately seemingly ready for use!

In [6]:
REinv.columns

Index(['Recipient Country/Area', 'ISO-code', 'Region', 'Project', 'Year',
       'Investor', 'Technology', 'Asset Class', 'Amount (USD million)',
       'Source'],
      dtype='object')

Renaming columns: 

In [7]:
REinv.rename(columns = {'Recipient Country/Area' : 'Country'}, inplace=True)

Sorting by year and country's alphabetical order:

In [8]:
REinv = REinv.sort_values(by = ['Country', 'Year'])

Transformed year into datetime format, pre-empting for future use: 

In [9]:
REinv.Year = pd.to_datetime(REinv.Year, format = '%Y')

In [10]:
REinv.Country.value_counts()

Brazil          779
Multilateral    427
India           383
Other Africa    308
China           240
               ... 
Anguilla          1
EU (28)           1
Cyprus            1
Russian Fed       1
Lithuania         1
Name: Country, Length: 191, dtype: int64

In [11]:
REinv.Country = pd.Series(REinv.Country)

In [12]:
REinv.describe()

,Country,ISO-code,Region,Project,Year,Investor,Technology,Asset Class,Amount (USD million),Source
count,9275,9275,9275,9023,9275,9275,9275,9275,9275,9275
unique,191,191,10,6696,18,25,8,10,6046,677
top,Brazil,BRA,Africa,TC AGGREGATED ACTIVITIES,2017-01-01 00:00:00,Others,Multiple renewables*,Grant,0.1000,2017 OECD Credit Report System (CRS) Database
freq,779,779,2775,588,1210,5344,3779,6359,29,979
first,NaN,NaN,NaN,NaN,2000-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN


In [13]:
REinv.rename(columns = {'Year': 'Date'}, inplace=True)

In [14]:
REinv.head()

,Country,ISO-code,Region,Project,Date,Investor,Technology,Asset Class,Amount (USD million),Source
0,Afghanistan,AFG,Asia,"ADA, MICRO HYDRO POWER",2000-01-01,Others,Renewable Hydropower,Grant,0.0225,2000-01 OECD Credit Report System (CRS) Database
33,Afghanistan,AFG,Asia,MICRO HYDRO POWER,2001-01-01,Others,Renewable Hydropower,Grant,0.0243,2000-01 OECD Credit Report System (CRS) Database
80,Afghanistan,AFG,Asia,SOLAR ENERGY PROJECT,2001-01-01,Others,Solar energy,Grant,0.0971,2000-01 OECD Credit Report System (CRS) Database
25,Afghanistan,AFG,Asia,HYDROELECTRIC POWER STATION I,2002-01-01,KFW,Renewable Hydropower,Grant,3.7018,2002-03 OECD Credit Report System (CRS) Database
2,Afghanistan,AFG,Asia,Afghanistan: Emergency National Solidarity Pro...,2003-01-01,WBG,Multiple renewables*,Grant,22.6498,2002-03 OECD Credit Report System (CRS) Database


In [2]:
#Dropping ISO-code & Region feature to only retain the Country's name:

In [15]:
REinv.drop(['ISO-code', 'Region'], axis=1, inplace=True)

Saving as a csv to recuperate for further use: 

In [16]:
irena.to_csv('irena_inv.csv', index=False)

Simplifying this feature's name to make it more readily extractable. This will be the target feature:

In [13]:
irena.rename(columns = {'Amount (USD million)': 'Invested_USDM'}, inplace=True)

Testing an alternative index, experimenting for better efficiency:

In [21]:
irena.set_index('Date', drop=True, inplace=True)

Creating a function to impute the Project column if empty, using as description merely its technology, so as to not lose valuable data contained in the observations with missing descriptions:  

In [5]:
def col_change(x):
    if type(x.Project)!= str:
        x.Project= x.Technology
    return x

In [14]:
irena = irena.apply(col_change, axis=1)

Transforming all words to lower case in order to prepare the description to be processed by a NLP:

In [15]:
irena.Project = irena.Project.apply(lambda x: x.lower())

Trial TFIDF to learn more about the Project Description [PD] column:

In [19]:

text_pj = irena_t.Project
text_tech = irena_t.Technology

tvec = TfidfVectorizer(stop_words='english', max_features = 500, ngram_range=(1, 2), strip_accents='unicode')

tvec.fit(text_pj)

text_csr1 = tvec.transform(text_pj)

pjtext_df = pd.DataFrame(text_csr1.todense(),
                  columns=tvec.get_feature_names())
pjtext_df.transpose().sort_values(0, ascending=False).transpose()
#Play around w/ max_features + min/max_df


,micro hydro,micro,hydro power,hydro,power,plant,planning,plan,pilot project,photovoltaique,...,energy projects,energy project,energy programme,energy program,energy production,energy generation,energy fund,energy environment,energy energy,wind project
0,0.585452,0.481332,0.446561,0.384796,0.279431,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.585452,0.481332,0.446561,0.384796,0.279431,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.662306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.269228,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9270,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9271,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9272,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9273,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
